In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [80]:
df = pd.read_csv("data/housing.csv")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [81]:
bin_cat = ["mainroad","guestroom","basement","hotwaterheating","airconditioning", "prefarea"]
df[bin_cat] = df[bin_cat].replace({"yes": 1, "no": 0})
df.head()

C:\Users\Mardon\AppData\Local\Temp\ipykernel_20124\3450382261.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[bin_cat] = df[bin_cat].replace({"yes": 1, "no": 0})


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [82]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(df["furnishingstatus"].to_numpy().reshape(-1,1))

OneHotEncoder()

In [83]:
df[encoder.get_feature_names_out(["status"])] = encoder.transform(df["furnishingstatus"].to_numpy().reshape(-1,1)).toarray().astype(int)
df.drop(["furnishingstatus"], axis=1, inplace=True)

In [84]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,status_furnished,status_semi-furnished,status_unfurnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1,0,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1,0,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1,0,0


In [85]:
df.isna().sum()

price                    0
area                     0
bedrooms                 0
bathrooms                0
stories                  0
mainroad                 0
guestroom                0
basement                 0
hotwaterheating          0
airconditioning          0
parking                  0
prefarea                 0
status_furnished         0
status_semi-furnished    0
status_unfurnished       0
dtype: int64

In [87]:
train_ratio = 0.8
split_idx = int(len(df)*train_ratio)
train_data = df[:split_idx]
test_data = df[split_idx:]
print(len(train_data))
print(len(test_data))

436
109


In [91]:
model = LinearRegression()
model.fit(train_data[[i for i in train_data.columns if i != "price"]], train_data["price"])

LinearRegression()

In [96]:
test_pred = data=model.predict(test_data[[i for i in train_data.columns if i != "price"]])
test_pred

array([4316123.25358721, 3922018.00649958, 2867042.87429815,
       2593219.93498831, 4008411.77477829, 3752907.83133628,
       3852735.2561097 , 3059447.30549261, 3177671.9913077 ,
       3097337.78976175, 4434284.76982441, 2952223.23155725,
       3817383.45795319, 2938943.32443233, 3999902.75323973,
       3887339.65391572, 5491746.48464026, 3170850.9916942 ,
       4552515.18341041, 3879307.14143553, 3542710.68371947,
       3259128.14201045, 3634041.80107903, 3000899.48174156,
       4320879.83443315, 4418298.44203518, 3199050.95255863,
       3494837.97909725, 4958787.29629275, 3069476.03765921,
       3097869.39891875, 4175076.61102144, 2992415.59919042,
       3468819.27330309, 2963787.06445075, 3611182.94910648,
       4641303.98152573, 4040977.24649254, 3619898.47555737,
       3474664.35652164, 4673460.87946892, 3334638.72054078,
       3339395.22286677, 3193242.057953  , 3455492.08356602,
       2528528.47558945, 2968057.22703597, 4409616.54118632,
       2389776.15243263,

In [101]:
test_price = np.array(test_data["price"])
mae = np.abs(test_price - test_pred).sum() / len(test_price)
mse = ((test_price - test_pred) ** 2).sum() / len(test_price)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print(model.score(test_data[[i for i in train_data.columns if i != "price"]], test_price))

Mean Absolute Error: 821318.9056217496
Mean Squared Error: 1048582305860.3655
-5.164065953933278
